In [ ]:
import json
import os
import glob
from sklearn.model_selection import train_test_split

def merge_json_files(input_dir, output_file):
    """
    Ghép tất cả các file JSON trong thư mục input_dir thành một file JSON duy nhất.
    
    Args:
        input_dir (str): Đường dẫn đến thư mục chứa các file JSON
        output_file (str): Đường dẫn đến file JSON đầu ra
    """
    merged_data = []
    
    # Tìm tất cả các file JSON trong thư mục
    json_files = glob.glob(os.path.join(input_dir, "*.json"))
    
    if not json_files:
        print("Không tìm thấy file JSON nào trong thư mục:", input_dir)
        return merged_data  # Trả về danh sách rỗng thay vì None
    
    # Đọc và ghép từng file JSON
    for json_file in json_files:
        if os.path.basename(json_file) in ["meta.json", "train.json", "test.json", "validation.json"]:
            continue
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                print(f"File {json_file} có kiểu dữ liệu: {type(data)}")
                # Chuẩn hóa dữ liệu: nếu không phải list, biến thành list
                if not isinstance(data, list):
                    data = [data]
                merged_data.extend(data)
        except json.JSONDecodeError as e:
            print(f"Lỗi khi đọc file {json_file}: {e}")
        except Exception as e:
            print(f"Lỗi không xác định khi xử lý file {json_file}: {e}")
    
    # Lưu dữ liệu ghép vào file đầu ra
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(merged_data, f, indent=4, ensure_ascii=False)
        print(f"Đã ghép thành công {len(json_files)} file JSON vào {output_file}")
    except Exception as e:
        print(f"Lỗi khi ghi file đầu ra {output_file}: {e}")
    
    return merged_data

def split_dataset(data, input_dir, train_ratio=0.7, test_ratio=0.15):
    """
    Chia dữ liệu thành 3 tập: train, test, và validation.
    
    Args:
        data (list): Dữ liệu cần chia
        input_dir (str): Thư mục để lưu các file train.json, test.json, validation.json
        train_ratio (float): Tỷ lệ tập train
        test_ratio (float): Tỷ lệ tập test (validation_ratio sẽ là 1 - train_ratio - test_ratio)
    """
    if not isinstance(data, list):
        print("Dữ liệu đầu vào phải là một danh sách!")
        return
    
    if len(data) == 0:
        print("Dữ liệu đầu vào rỗng, không thể chia tập!")
        return
    
    train_data, temp_data = train_test_split(data, train_size=train_ratio, random_state=42)
    validation_ratio = (1 - train_ratio - test_ratio) / (1 - train_ratio)
    test_data, validation_data = train_test_split(temp_data, train_size=(test_ratio / (test_ratio + (1 - train_ratio - test_ratio))), random_state=42)
    
    for dataset, filename in [(train_data, "train.json"), (test_data, "test.json"), (validation_data, "validation.json")]:
        output_path = os.path.join(input_dir, filename)
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(dataset, f, indent=4, ensure_ascii=False)
            print(f"Đã lưu {filename} với {len(dataset)} mẫu")
        except Exception as e:
            print(f"Lỗi khi ghi file {filename}: {e}")

if __name__ == "__main__":
    input_directory = "./json"
    output_dir = "./dataset/"
    output_file = os.path.join(output_dir, "meta.json")
    
    merged_data = merge_json_files(input_directory, output_file)
    print(f"Kiểu dữ liệu của merged_data: {type(merged_data)}")
    
    if merged_data is not None:
        split_dataset(merged_data, output_dir, train_ratio=0.7, test_ratio=0.15)
    else:
        print("Không thể ghép dữ liệu, merged_data là None!")                   

In [ ]:
import json
import os
import glob
from sklearn.model_selection import train_test_split

def merge_json_files(input_dir, output_file):
    """
    Ghép tất cả các file JSON trong thư mục input_dir thành một file JSON duy nhất.
    
    Args:
        input_dir (str): Đường dẫn đến thư mục chứa các file JSON
        output_file (str): Đường dẫn đến file JSON đầu ra
    """
    merged_data = []
    
    # Tìm tất cả các file JSON trong thư mục
    json_files = glob.glob(os.path.join(input_dir, "*.json"))
    
    if not json_files:
        print("Không tìm thấy file JSON nào trong thư mục:", input_dir)
        return merged_data
    
    # Đọc và ghép từng file JSON
    for json_file in json_files:
        if os.path.basename(json_file) in ["meta.json", "train.json", "test.json", "validation.json"]:
            continue
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                print(f"File {json_file} có kiểu dữ liệu: {type(data)}")
                if not isinstance(data, list):
                    data = [data]
                merged_data.extend(data)
        except json.JSONDecodeError as e:
            print(f"Lỗi khi đọc file {json_file}: {e}")
        except Exception as e:
            print(f"Lỗi không xác định khi xử lý file {json_file}: {e}")
    
    # Lưu dữ liệu ghép vào file đầu ra
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(merged_data, f, indent=4, ensure_ascii=False)
        print(f"Đã ghép thành công {len(json_files)} file JSON vào {output_file}")
    except Exception as e:
        print(f"Lỗi khi ghi file đầu ra {output_file}: {e}")
    
    return merged_data

def update_image_paths(meta_file, base_dir):
    """
    Cập nhật đường dẫn tuyệt đối trong trường 'image' của meta.json thành đường dẫn tương đối.
    
    Args:
        meta_file (str): Đường dẫn đến file meta.json
        base_dir (str): Thư mục hiện tại để tính đường dẫn tương đối
    Returns:
        list: Dữ liệu đã cập nhật
    """
    try:
        # Đọc file meta.json
        with open(meta_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Duyệt qua từng mục và cập nhật trường 'image'
        for item in data:
            if "image" in item:
                abs_path = item["image"]
                rel_path = os.path.relpath(abs_path, base_dir)
                item["image"] = rel_path.replace("\\", "/")
            
        # Lưu lại file meta.json
        with open(meta_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"Đã cập nhật đường dẫn trong {meta_file}")
        return data
    except Exception as e:
        print(f"Lỗi khi cập nhật đường dẫn trong {meta_file}: {e}")
        return []

def split_dataset(data, input_dir, train_ratio=0.7, test_ratio=0.15):
    """
    Chia dữ liệu thành 3 tập: train, test, và validation.
    
    Args:
        data (list): Dữ liệu cần chia
        input_dir (str): Thư mục để lưu các file train.json, test.json, validation.json
        train_ratio (float): Tỷ lệ tập train
        test_ratio (float): Tỷ lệ tập test
    """
    if not isinstance(data, list):
        print("Dữ liệu đầu vào phải là một danh sách!")
        return
    
    if len(data) == 0:
        print("Dữ liệu đầu vào rỗng, không thể chia tập!")
        return
    
    train_data, temp_data = train_test_split(data, train_size=train_ratio, random_state=42)
    validation_ratio = (1 - train_ratio - test_ratio) / (1 - train_ratio)
    test_data, validation_data = train_test_split(temp_data, train_size=(test_ratio / (test_ratio + (1 - train_ratio - test_ratio))), random_state=42)
    
    for dataset, filename in [(train_data, "train.json"), (test_data, "test.json"), (validation_data, "validation.json")]:
        output_path = os.path.join(input_dir, filename)
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(dataset, f, indent=4, ensure_ascii=False)
            print(f"Đã lưu {filename} với {len(dataset)} mẫu")
        except Exception as e:
            print(f"Lỗi khi ghi file {filename}: {e}")

if __name__ == "__main__":
    input_directory = "./json"
    output_dir = "./dataset/"
    output_file = os.path.join(output_dir, "meta.json")
    
    # Ghép các file JSON
    merged_data = merge_json_files(input_directory, output_file)
    print(f"Kiểu dữ liệu của merged_data: {type(merged_data)}")
    
    if merged_data:
        # Cập nhật đường dẫn và lấy dữ liệu đã sửa
        updated_data = update_image_paths(output_file, output_dir)
        if updated_data:
            # Chia tập dữ liệu với dữ liệu đã cập nhật
            split_dataset(updated_data, output_dir, train_ratio=0.7, test_ratio=0.15)
        else:
            print("Không thể lấy dữ liệu đã cập nhật, bỏ qua bước chia tập!")
    else:
        print("Không thể ghép dữ liệu, merged_data rỗng!")

In [ ]:
import json
import os
import glob
from sklearn.model_selection import train_test_split

def merge_json_files(input_dir, output_file):
    """
    Ghép tất cả các file JSON trong thư mục input_dir thành một file JSON duy nhất.
    
    Args:
        input_dir (str): Đường dẫn đến thư mục chứa các file JSON
        output_file (str): Đường dẫn đến file JSON đầu ra
    """
    merged_data = []
    
    json_files = glob.glob(os.path.join(input_dir, "*.json"))
    
    if not json_files:
        print("Không tìm thấy file JSON nào trong thư mục:", input_dir)
        return merged_data
    
    for json_file in json_files:
        if os.path.basename(json_file) in ["meta.json", "train.json", "test.json", "validation.json"]:
            continue
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                print(f"File {json_file} có kiểu dữ liệu: {type(data)}")
                if not isinstance(data, list):
                    data = [data]
                merged_data.extend(data)
        except json.JSONDecodeError as e:
            print(f"Lỗi khi đọc file {json_file}: {e}")
        except Exception as e:
            print(f"Lỗi không xác định khi xử lý file {json_file}: {e}")
    
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(merged_data, f, indent=4, ensure_ascii=False)
        print(f"Đã ghép thành công {len(json_files)} file JSON vào {output_file}")
    except Exception as e:
        print(f"Lỗi khi ghi file đầu ra {output_file}: {e}")
    
    return merged_data

def update_image_paths(meta_file, base_dir):
    """
    Cập nhật đường dẫn tuyệt đối trong trường 'image' thành đường dẫn tương đối.
    
    Args:
        meta_file (str): Đường dẫn đến file meta.json
        base_dir (str): Thư mục hiện tại để tính đường dẫn tương đối
    Returns:
        list: Dữ liệu đã cập nhật
    """
    try:
        with open(meta_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        for item in data:
            if "image" in item:
                abs_path = item["image"]
                rel_path = os.path.relpath(abs_path, base_dir)
                item["image"] = rel_path.replace("\\", "/")
            
        with open(meta_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"Đã cập nhật đường dẫn trong {meta_file}")
        return data
    except Exception as e:
        print(f"Lỗi khi cập nhật đường dẫn trong {meta_file}: {e}")
        return []

def split_dataset(data, output_dir, train_ratio=0.7, test_ratio=0.15):
    """
    Chia dữ liệu thành 3 tập: train, test, và validation, lưu dưới dạng metadata.jsonl.
    
    Args:
        data (list): Dữ liệu cần chia
        output_dir (str): Thư mục để lưu các thư mục train, test, validation
        train_ratio (float): Tỷ lệ tập train
        test_ratio (float): Tỷ lệ tập test
    """
    if not isinstance(data, list):
        print("Dữ liệu đầu vào phải là một danh sách!")
        return
    
    if len(data) == 0:
        print("Dữ liệu đầu vào rỗng, không thể chia tập!")
        return
    
    # Chia dữ liệu
    train_data, temp_data = train_test_split(data, train_size=train_ratio, random_state=42)
    validation_ratio = (1 - train_ratio - test_ratio) / (1 - train_ratio)
    test_data, validation_data = train_test_split(temp_data, train_size=(test_ratio / (test_ratio + (1 - train_ratio - test_ratio))), random_state=42)
    
    # Tạo thư mục và lưu dữ liệu dưới dạng JSONL
    for dataset, split_name in [(train_data, "train"), (test_data, "test"), (validation_data, "validation")]:
        split_dir = os.path.join(output_dir, split_name)
        os.makedirs(split_dir, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại
        output_path = os.path.join(split_dir, "metadata.jsonl")
        
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                for item in dataset:
                    # Ánh xạ dữ liệu sang định dạng JSONL
                    jsonl_item = {
                        "file_name": item.get("image", ""),
                        "ground_truth": json.dumps(item.get("ground_truth", {"gt_parse": {}}), ensure_ascii=False)
                    }
                    f.write(json.dumps(jsonl_item, ensure_ascii=False) + "\n")
            print(f"Đã lưu {split_name}/metadata.jsonl với {len(dataset)} mẫu")
        except Exception as e:
            print(f"Lỗi khi ghi file {output_path}: {e}")

if __name__ == "__main__":
    input_directory = r"D:\Private\Project\kaggle\LabAI\Donut\donut\json"
    output_dir = r"D:\Private\Project\kaggle\LabAI\Donut\donut\dataset"
    output_file = os.path.join(output_dir, "meta.json")
    
    # Ghép các file JSON
    merged_data = merge_json_files(input_directory, output_file)
    print(f"Kiểu dữ liệu của merged_data: {type(merged_data)}")
    
    if merged_data:
        # Cập nhật đường dẫn và lấy dữ liệu đã sửa
        updated_data = update_image_paths(output_file, output_dir)
        if updated_data:
            # Chia tập dữ liệu với dữ liệu đã cập nhật
            split_dataset(updated_data, output_dir, train_ratio=0.7, test_ratio=0.15)
            # Xóa file meta.json
            try:
                os.remove(output_file)
                print(f"Đã xóa file {output_file}")
            except Exception as e:
                print(f"Lỗi khi xóa file {output_file}: {e}")
        else:
            print("Không thể lấy dữ liệu đã cập nhật, bỏ qua bước chia tập và xóa file!")
    else:
        print("Không thể ghép dữ liệu, merged_data rỗng!")

In [7]:
import json
import os
import glob
import shutil
from sklearn.model_selection import train_test_split

def merge_json_files(input_dir, output_file):
    """
    Ghép tất cả các file JSON trong thư mục input_dir thành một file JSON duy nhất.
    
    Args:
        input_dir (str): Đường dẫn đến thư mục chứa các file JSON
        output_file (str): Đường dẫn đến file JSON đầu ra
    """
    merged_data = []
    
    json_files = glob.glob(os.path.join(input_dir, "*.json"))
    
    if not json_files:
        print("Không tìm thấy file JSON nào trong thư mục:", input_dir)
        return merged_data
    
    for json_file in json_files:
        if os.path.basename(json_file) in ["meta.json", "train.json", "test.json", "validation.json"]:
            continue
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                print(f"File {json_file} có kiểu dữ liệu: {type(data)}")
                if not isinstance(data, list):
                    data = [data]
                merged_data.extend(data)
        except json.JSONDecodeError as e:
            print(f"Lỗi khi đọc file {json_file}: {e}")
        except Exception as e:
            print(f"Lỗi không xác định khi xử lý file {json_file}: {e}")
    
    try:
        # Tạo thư mục chứa output_file nếu chưa tồn tại
        output_dir = os.path.dirname(output_file)
        os.makedirs(output_dir, exist_ok=True)
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(merged_data, f, indent=4, ensure_ascii=False)
        print(f"Đã ghép thành công {len(json_files)} file JSON vào {output_file}")
    except Exception as e:
        print(f"Lỗi khi ghi file đầu ra {output_file}: {e}")
    
    return merged_data

def update_image_paths(meta_file, images_dir):
    """
    Cập nhật trường 'image' để chứa đường dẫn chính xác trong images_dir.
    
    Args:
        meta_file (str): Đường dẫn đến file meta.json
        images_dir (str): Thư mục chứa các file ảnh (/kaggle/working/nghia_donut/data/images/)
    Returns:
        list: Dữ liệu đã cập nhật
    """
    try:
        with open(meta_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        for item in data:
            if "image" in item:
                # Chuẩn hóa đường dẫn và lấy tên file ảnh
                image_path = item["image"].replace("\\", "/")
                image_filename = os.path.basename(image_path)
                # Tạo đường dẫn đầy đủ trong images_dir
                full_image_path = os.path.join(images_dir, image_filename)
                # Cập nhật trường image để chứa đường dẫn đầy đủ
                item["image"] = full_image_path
        
        with open(meta_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"Đã cập nhật đường dẫn trong {meta_file}")
        return data
    except Exception as e:
        print(f"Lỗi khi cập nhật đường dẫn trong {meta_file}: {e}")
        return []

def split_dataset(data, output_dir, train_ratio=0.7, test_ratio=0.15):
    """
    Chia dữ liệu thành 3 tập: train, test, và validation, lưu dưới dạng metadata.jsonl.
    Sao chép các file ảnh vào thư mục tương ứng và cập nhật file_name.
    
    Args:
        data (list): Dữ liệu cần chia
        output_dir (str): Thư mục để lưu các thư mục train, test, validation
        train_ratio (float): Tỷ lệ tập train
        test_ratio (float): Tỷ lệ tập test
    """
    if not isinstance(data, list):
        print("Dữ liệu đầu vào phải là một danh sách!")
        return
    
    if len(data) == 0:
        print("Dữ liệu đầu vào rỗng, không thể chia tập!")
        return
    
    # Chia dữ liệu
    train_data, temp_data = train_test_split(data, train_size=train_ratio, random_state=42)
    validation_ratio = (1 - train_ratio - test_ratio) / (1 - train_ratio)
    test_data, validation_data = train_test_split(temp_data, train_size=(test_ratio / (test_ratio + (1 - train_ratio - test_ratio))), random_state=42)
    
    # Tạo thư mục và lưu dữ liệu dưới dạng JSONL
    for dataset, split_name in [(train_data, "train"), (test_data, "test"), (validation_data, "validation")]:
        split_dir = os.path.join(output_dir, split_name)
        os.makedirs(split_dir, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại
        output_path = os.path.join(split_dir, "metadata.jsonl")
        
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                for item in dataset:
                    # Lấy đường dẫn ảnh (đã được cập nhật trong update_image_paths)
                    image_path = item.get("image", "")
                    if not image_path or not os.path.exists(image_path):
                        print(f"Cảnh báo: File ảnh {image_path} không tồn tại, bỏ qua mục này")
                        continue
                    
                    # Lấy tên file ảnh
                    image_filename = os.path.basename(image_path)
                    # Đường dẫn đích để sao chép file ảnh
                    dest_image_path = os.path.join(split_dir, image_filename)
                    
                    # Sao chép file ảnh vào thư mục split
                    try:
                        shutil.copy2(image_path, dest_image_path)
                        print(f"Đã sao chép {image_path} đến {dest_image_path}")
                    except Exception as e:
                        print(f"Lỗi khi sao chép file ảnh {image_path}: {e}")
                        continue
                    
                    # Ánh xạ dữ liệu sang định dạng JSONL
                    jsonl_item = {
                        "file_name": image_filename,  # Chỉ sử dụng tên file
                        "ground_truth": json.dumps(item.get("ground_truth", {"gt_parse": {}}), ensure_ascii=False)
                    }
                    f.write(json.dumps(jsonl_item, ensure_ascii=False) + "\n")
            print(f"Đã lưu {split_name}/metadata.jsonl với {len(dataset)} mẫu")
        except Exception as e:
            print(f"Lỗi khi ghi file {output_path}: {e}")

if __name__ == "__main__":
    # Đường dẫn đến thư mục chứa các file JSON
    input_directory = "D:/Private/Project/kaggle/LabAI/Donut/donut/json"
    # Đường dẫn đến thư mục dataset
    output_dir = "D:/Private/Project/kaggle/LabAI/Donut/donut/dataset"
    # Đường dẫn đến thư mục chứa file ảnh gốc
    images_dir = "D:/Private/Project/kaggle/LabAI/Donut/donut/data/images"
    # Đường dẫn đến file meta.json
    output_file = os.path.join(output_dir, "meta.json")
    
    # Ghép các file JSON
    merged_data = merge_json_files(input_directory, output_file)
    print(f"Kiểu dữ liệu của merged_data: {type(merged_data)}")
    
    if merged_data:
        # Cập nhật đường dẫn và lấy dữ liệu đã sửa
        updated_data = update_image_paths(output_file, images_dir)
        if updated_data:
            # Chia tập dữ liệu với dữ liệu đã cập nhật
            split_dataset(updated_data, output_dir, train_ratio=0.7, test_ratio=0.15)
            # Xóa file meta.json
            try:
                os.remove(output_file)
                print(f"Đã xóa file {output_file}")
            except Exception as e:
                print(f"Lỗi khi xóa file {output_file}: {e}")
        else:
            print("Không thể lấy dữ liệu đã cập nhật, bỏ qua bước chia tập và xóa file!")
    else:
        print("Không thể ghép dữ liệu, merged_data rỗng!")

File D:/Private/Project/kaggle/LabAI/Donut/donut/json\MBbank.json có kiểu dữ liệu: <class 'list'>
File D:/Private/Project/kaggle/LabAI/Donut/donut/json\SaigonCommercialBank.json có kiểu dữ liệu: <class 'list'>
File D:/Private/Project/kaggle/LabAI/Donut/donut/json\Techbank.json có kiểu dữ liệu: <class 'list'>
File D:/Private/Project/kaggle/LabAI/Donut/donut/json\Vietcombank.json có kiểu dữ liệu: <class 'list'>
Đã ghép thành công 4 file JSON vào D:/Private/Project/kaggle/LabAI/Donut/donut/dataset\meta.json
Kiểu dữ liệu của merged_data: <class 'list'>
Đã cập nhật đường dẫn trong D:/Private/Project/kaggle/LabAI/Donut/donut/dataset\meta.json
Đã sao chép D:/Private/Project/kaggle/LabAI/Donut/donut/data/images\page_MBbank_34.jpg đến D:/Private/Project/kaggle/LabAI/Donut/donut/dataset\train\page_MBbank_34.jpg
Đã sao chép D:/Private/Project/kaggle/LabAI/Donut/donut/data/images\page_Techbank_3.jpg đến D:/Private/Project/kaggle/LabAI/Donut/donut/dataset\train\page_Techbank_3.jpg
Đã sao chép D:/Pr

In [8]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

In [9]:
!python test.py --dataset_name_or_path ./dataset --pretrained_model_name_or_path ./result_new/train_cord/test_experiment --save_path ./result/output.json


Total number of samples: 16, Tree Edit Distance (TED) based accuracy score: 0.019463551499399913, F1 accuracy score: 0.005638340716874748


c:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]




Generating train split: 0 examples [00:00, ? examples/s]
Generating train split: 75 examples [00:00, 4044.39 examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]
Generating validation split: 17 examples [00:00, 1353.77 examples/s]

Generating test split: 0 examples [00:00, ? examples/s]
Generating test split: 16 examples [00:00, 1260.52 examples/s]

100%|██████████| 16/16 [00:38<00:00,  2.44s/it]


In [10]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch

# Tải processor và mô hình
processor = DonutProcessor.from_pretrained("result_new/config/my_experiment")
model = VisionEncoderDecoderModel.from_pretrained("result_new/config/my_experiment")

# Tải và tiền xử lý ảnh
image = Image.open(r"D:\Private\Project\kaggle\LabAI\Donut\donut\dataset\train\page_MBbank_2.jpg").convert("RGB")
pixel_values = processor(image, return_tensors="pt").pixel_values

# Chuyển sang GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
pixel_values = pixel_values.to(device)

# Tạo dự đoán
model.eval()
with torch.no_grad():
    task_prompt = "<s_dataset>"  # Điều chỉnh nếu cần
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=128,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        num_beams=1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )

# Giải mã đầu ra
decoded_output = processor.tokenizer.batch_decode(outputs.sequences)[0]
print("Dự đoán:", decoded_output)

c:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: Can't load image processor for 'result_new/config/my_experiment'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'result_new/config/my_experiment' is the correct path to a directory containing a preprocessor_config.json file